# EXPLANATION
This script performs GWAS on a pre-selected number of variants (in Plink binary format) and on a specific phenotype you want. Ideally, the phenotype has been adjusted for inversions, infection, and also log-transformed if the raw phenotype's errors weren't normally distributed.

BEWARE: You should format your phenotype line IDs in the same way as denoted in the .fam variant file you are feeding.

 # Time measurement


In [1]:
import time
start = time.clock()

# CONSTANTS and INPUTS 

In [2]:
import os
import pandas as pd

data = pd.read_csv('Inputs/GWAS_Constants.txt', header=None)

NUMBER_OF_PERMUTATIONS = int(data[0][0])
OUTPUT_NAME = data[0][1]
PHENOTYPE_DATA = data[0][2]
VARIANTS_TO_TEST = data[0][3]
USE_OFFICIAL_GSM = data[0][4]
if USE_OFFICIAL_GSM == "TRUE":
    USE_OFFICIAL_GSM = True
else:
    USE_OFFICIAL_GSM = False
    
    
print 'Time (s): ' + str(time.clock()-start)

Time (s): 0.331671272727


# Importing, general preparations

In [3]:
from fastlmm.association import single_snp
from fastlmm.inference.fastlmm_predictor import _snps_fixup, _pheno_fixup, _kernel_fixup, _SnpTrainTest
from random import shuffle
import numpy as np
import time
import os
# We're going to need PySnpTools, to do permutations, because we can shuffle bed files by varint using Bed
import sys
sys.path.append('../PySnpTools')
from pysnptools.snpreader import Bed
from shutil import copyfile

print 'Time (s): ' + str(time.clock()-start)

Time (s): 1.48057939394


C:\Users\micha\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Actual GWAS

In [4]:
# Clearing cache
# This ensures that the relationship matrix is recalculated for each phenotype.
try:
    os.remove('Outputs/Fast-Lmm-Cache/Gwas-Permutations-Cache.npz')
except OSError:
    pass



# Performing GWAS on the real phenotype:

if USE_OFFICIAL_GSM == False:
    results_df = single_snp(VARIANTS_TO_TEST,  PHENOTYPE_DATA,
                            cache_file='Outputs/Fast-Lmm-Cache/Gwas-Permutations-Cache.npz',
                            leave_out_one_chrom=False,
                            save_test_statistic=True,
                            output_file_name = 'Outputs/' + OUTPUT_NAME + '_Original.txt',
                            )
else:
    
    # First, constructing GSM form official relationship matrix (GSM):
    import re
    import sys
    from pysnptools.kernelreader import KernelData

    with open('Raw_Data/freeze2.common.rel.mat') as f:
        first_line = f.readline()


    tmp = re.split(r'\t+', first_line.rstrip())
    tmp = tmp[1:]
    # print tmp
    iids = []
    for i in range(len(tmp)):
         iids.append(re.split(r' +', tmp[i]))

    Gsm = np.genfromtxt('Raw_Data/freeze2.common.rel.mat', skip_header=True)
    # print Gsm

    Gsm = Gsm[:, 2:207]
    # Gsm = LMM(K=Gsm)
    # Gsm.setSU_fromK()
    # Gsm.K

    # np.savetxt('../Inputs/NCSU_GSM_U.txt', Gsm.U)
    # np.savetxt('../Inputs/NCSU_GSM_S.txt', Gsm.S)
    
    my_kernel = KernelData(iid=iids, val=Gsm.tolist())
    
    # Now, performing GSM using the official GSM as kernel:
    results_df = single_snp(VARIANTS_TO_TEST,
                            PHENOTYPE_DATA,
                            K0=my_kernel,
                            leave_out_one_chrom=False,
                            save_test_statistic=True,
                            output_file_name = 'Outputs/' + OUTPUT_NAME + '_Original.txt',
                            )

print 'Total Time (s): ' + str(time.clock()-start)

C:\Users\micha\Anaconda2\lib\site-packages\pysnptools-0.3.13-py2.7-win-amd64.egg\pysnptools\snpreader\bed.py:42: FutureWarning: 'count_A1' was not set. For now it will default to 'False', but in the future it will default to 'True'
  warnings.warn("'count_A1' was not set. For now it will default to 'False', but in the future it will default to 'True'", FutureWarning)
C:\Users\micha\Anaconda2\lib\site-packages\pysnptools-0.3.13-py2.7-win-amd64.egg\pysnptools\snpreader\snpreader.py:625: FutureWarning: Conversion of the second argument of issubdtype from `str` to `str` is deprecated. In future, it will be treated as `np.string_ == np.dtype(str).type`.
  assert np.issubdtype(self._row.dtype, str) and len(self._row.shape)==2 and self._row.shape[1]==2, "iid should be dtype str, have two dimensions, and the second dimension should be size 2"
C:\Users\micha\Anaconda2\lib\site-packages\pysnptools-0.3.13-py2.7-win-amd64.egg\pysnptools\snpreader\snpreader.py:626: FutureWarning: Conversion of the 

Total Time (s): 67.186918303


In [5]:
test_stat = pd.read_csv('Outputs/Fast-Lmm-Cache/Test-Stat-Cache.txt', header=None)
test_stat = test_stat.replace('[\[\] ]', '', regex=True)
test_stat = pd.to_numeric(test_stat[0])

results_df['Full ID'] = results_df['Chr'].astype('str') + '_' + results_df['ChrPos'].astype('str')
results_df = pd.concat([results_df[['Chr', 'ChrPos', 'SNP', 'Full ID', 'PValue']], test_stat],
                       axis = 1)
results_df.columns = ['Chr', 'ChrPos', 'SNP', 'Full ID', 'PValue', 'F-test statistic']

mybed = Bed(VARIANTS_TO_TEST + '.bed')
mysnpdata = mybed.read()

print 'Time (s): ' + str(time.clock()-start)

Time (s): 68.8176252121


In [6]:
pheno = _pheno_fixup(PHENOTYPE_DATA, count_A1=None).read()
pheno = pheno.val[np.searchsorted(pheno.iid[:,1], mysnpdata.iid[:,1])]
snpdata = mysnpdata.val
diff = range(snpdata.shape[1])
maf = range(snpdata.shape[1])
n_alleles = range(snpdata.shape[1])
mean_major = range(snpdata.shape[1])
for i in range(snpdata.shape[1]):
    ref = [j for j, x in enumerate(snpdata[:,i]) if x == 2]
    alt = [j for j, x in enumerate(snpdata[:,i]) if x == 0]
    meanref = np.mean(pheno[ref])
    meanalt = np.mean(pheno[alt])
    if len(ref) > len(alt):
        diff[i] = meanref - meanalt
        maf[i] = float(len(alt)) / (len(ref) + len(alt))
        n_alleles[i] = len(ref) + len(alt)
        mean_major[i] = meanref
    elif len(ref) + len(alt) == 0:
        diff[i] = float('NaN')
        maf[i] = float('NaN')
        n_alleles[i] = len(ref) + len(alt)
        mean_major[i] = float('NaN')
    else:
        diff[i] = meanalt - meanref
        maf[i] = float(len(ref)) / (len(ref) + len(alt))
        n_alleles[i] = len(ref) + len(alt)
        mean_major[i] = meanalt

In [7]:
diff_df = diff_df = pd.DataFrame(data={'MajMinDiff':diff,
                                       'MeanMajor': mean_major,
                                       'MAF':maf,
                                       'NAlleles':n_alleles})
diff_df['SNP'] = mysnpdata.sid
results_df = pd.merge(results_df, diff_df, on='SNP')
    

# PHENOTYPE shuffling/permutation and adding the p-values from the resulting
# GWAS to the results data frame.
#phenotype_to_shuffle = pd.read_table(PHENOTYPE_DATA,
#                                     sep=' ', header=None)
#indices = range(len(phenotype_to_shuffle))
#temp_shuffled_pheno = '../Outputs/Fast-Lmm-Inputs/Temporary-Shuffled-Phenotype.txt'
#
#for i in range(NUMBER_OF_PERMUTATIONS):
#    time_permut_0 = time.time()
#    shuffle(indices)
#    phenotype_shuffled = []
#    for j in range(len(indices)):
#        phenotype_shuffled.append(phenotype_to_shuffle[2][indices[j]])
#    
#    phenotype_to_shuffle[2] = phenotype_shuffled
#    phenotype_to_shuffle.to_csv(temp_shuffled_pheno, header=False, index=False, sep=' ')
#    tmp_shuffled_df = single_snp(VARIANTS_TO_TEST,  temp_shuffled_pheno,
##                                cache_file='../Outputs/Fast-Lmm-Cache/Gwas-Permutations-Cache'+str(i)
#                                 cache_file='../Outputs/Fast-Lmm-Cache/Gwas-Permutations-Cache.npz',
#                                 leave_out_one_chrom=False,
#                                 )
#    tmp_shuffled_df['Full ID'] = tmp_shuffled_df['Chr'].astype('str') + '_' + tmp_shuffled_df['ChrPos'].astype('str')
#    
#    # sorting the new df to match the original
#    tmp_shuffled_df = tmp_shuffled_df[['Full ID', 'PValue']]
#    tmp_shuffled_df['PValue'].rename('PValueShuffled'+str(i+1))
#    
#    
#    results_df = pd.merge(results_df, tmp_shuffled_df, on='Full ID')
#    print('Time for permutation GWAS:' + str(time.time() - time_permut_0) + 's')

# Permutations

In [8]:
# Shuffling ALLELES by VARIANT

for i in range(NUMBER_OF_PERMUTATIONS):
    time_permut_0 = time.time()
    
    # Python works a little different than R: Shuffle directly modifies the input data frame!
    np.random.shuffle(mysnpdata.val)
    Bed.write('VariantsPermuted', mysnpdata)
    copyfile(VARIANTS_TO_TEST + '.bim', 'VariantsPermuted.bim')

    tmp_shuffled_df = single_snp('VariantsPermuted',  PHENOTYPE_DATA,
#                                cache_file='Outputs/Fast-Lmm-Cache/Gwas-Permutations-Cache'+str(i)
                                 cache_file='Outputs/Fast-Lmm-Cache/Gwas-Permutations-Cache.npz',
                                 leave_out_one_chrom=False,
                                 )
    tmp_shuffled_df['Full ID'] = tmp_shuffled_df['Chr'].astype('str') + '_' + tmp_shuffled_df['ChrPos'].astype('str')
    
    # sorting the new df to match the original
    tmp_shuffled_df = tmp_shuffled_df[['Full ID', 'SNP', 'PValue']]
    tmp_shuffled_df = tmp_shuffled_df.rename(columns={'Full ID':'Full IDShuffled'+str(i+1),
                                                      'PValue':'PValueShuffled'+str(i+1)})
    
    snpdata = mysnpdata.val
    diff = range(snpdata.shape[1])
    maf = range(snpdata.shape[1])
    n_alleles = range(snpdata.shape[1])
    mean_major = range(snpdata.shape[1])
    for k in range(snpdata.shape[1]):
        ref = [j for j, x in enumerate(snpdata[:,k]) if x == 2]
        alt = [j for j, x in enumerate(snpdata[:,k]) if x == 0]
        meanref = np.mean(pheno[ref])
        meanalt = np.mean(pheno[alt])
        if len(ref) > len(alt):
            diff[k] = meanref - meanalt
            maf[k] = float(len(alt)) / (len(ref) + len(alt))
            n_alleles[k] = len(ref) + len(alt)
            mean_major[k] = meanref
        elif len(ref) + len(alt) == 0:
            diff[k] = float('NaN')
            maf[k] = float('NaN')
            n_alleles[k] = len(ref) + len(alt)
            mean_major[k] = float('NaN')
        else:
            diff[k] = meanalt - meanref
            maf[k] = float(len(ref)) / (len(ref) + len(alt))
            n_alleles[k] = len(ref) + len(alt)
            mean_major[k] = meanalt
        
    diff_df = diff_df = pd.DataFrame(data={'MajMinDiffShuffled'+str(i+1):diff,
                                           'MeanMajorShuffled'+str(i+1): mean_major,
                                           'NAllelesShuffled'+str(i+1):n_alleles,
                                           'MAFShuffled'+str(i+1):maf})
    diff_df['SNP'] = mysnpdata.sid
    tmp_shuffled_df = pd.merge(tmp_shuffled_df, diff_df, on='SNP')
    tmp_shuffled_df = tmp_shuffled_df.rename(columns={'SNP':'SNPShuffled'+str(i+1)})
    
#    results_df = pd.merge(results_df, tmp_shuffled_df, on='Full ID')
    results_df = results_df.join(tmp_shuffled_df)
    print('Time for permutation GWAS:' + str(time.time() - time_permut_0) + 's')   

In [9]:
results_df.to_csv(OUTPUT_NAME + '_with_Permutations.txt', sep="\t", index=False)

# Manhattan Plot

In [15]:
import matplotlib.pyplot as plt
# makes sure can save pictures that a display is present!
plt.switch_backend('agg')

import pylab
import fastlmm.util.util as flutil

# better manhattan plot than MS Genomics one (or at least more robust)
# can be tweaked at will!
from assocplots.manhattan import *
from matplotlib.pyplot import figure


figure(num=None, figsize=(10, 7), dpi=80,)

manhattan(results_df["PValue"], results_df["ChrPos"], results_df["Chr"], OUTPUT_NAME,
         lines=[5], colors=['r', 'b'], cut=1)

plt.savefig('Figures/' + OUTPUT_NAME + '_Manhattan_Plot.png')
plt.close()

# QQ Plot

In [11]:
from fastlmm.util.stats import plotp
plotp.qqplot(results_df["PValue"].values, xlim=[0,5], ylim=[0,5])
pylab.savefig('Figures/' + OUTPUT_NAME + 'QQ_Plot.png')
pylab.close()

print 'Time after QQ plot: ' + str(time.clock()-start)

C:\Users\micha\Anaconda2\lib\site-packages\sklearn\grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
C:\Users\micha\Anaconda2\lib\site-packages\fastlmm-0.2.31-py2.7-win-amd64.egg\fastlmm\feature_selection\PerformSelectionDistributable.py:13: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "C:\Users\micha\Anaconda2\lib\runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "C:\Users\micha\Anaconda2\lib\runpy.py", line 72, in _run_code
    exec code in run_globals
  File "C:\Users\

lambda=0.9961


C:\Users\micha\Anaconda2\lib\site-packages\numpy\lib\scimath.py:120: RuntimeWarning: invalid value encountered in less
  if any(isreal(x) & (x < 0)):


Time after QQ plot: 108.641943273


# The End

In [12]:
print "\nDone!\n\n"


Done!


